## Домашнее задание 1

    База про спортсменов

    Креативная часть: найти парамеры и комбинации их, которые будут увеличивать accuracy

    Техническая часть: сделать под это всё удобные обёртки. Например, это функция, которая просто принимает список фичей, а на выходе показывает результат фита


In [57]:
import pandas as pd
import numpy as np
from sklearn import model_selection, tree, svm
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection, metrics
from sklearn import datasets, linear_model, model_selection


import pandas as pd
from sklearn import datasets, linear_model, model_selection
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix



data = pd.read_csv(r'/Users/olgahromova/Desktop/УчЕБА/МИСиС ДЗ по Python/athletes_sochi.csv').dropna()
data["BMI"] = data["weight"] / (data["height"]**2)
data

,age,birthdate,gender,height,name,weight,gold_medals,silver_medals,bronze_medals,total_medals,sport,country,BMI
0,17,1996-04-12,Male,1.72,Aaron Blunck,68.0,0,0,0,0,Freestyle Skiing,United States,22.985398
1,27,1986-05-14,Male,1.85,Aaron March,85.0,0,0,0,0,Snowboard,Italy,24.835646
2,21,1992-06-30,Male,1.78,Abzal Azhgaliyev,68.0,0,0,0,0,Short Track,Kazakhstan,21.461937
4,21,1992-07-30,Male,1.86,Adam Barwood,82.0,0,0,0,0,Alpine Skiing,New Zealand,23.702162
5,21,1992-12-18,Male,1.75,Adam Cieslar,57.0,0,0,0,0,Nordic Combined,Poland,18.612245
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2851,28,1985-04-30,Male,1.93,Ziga Pavlin,98.0,0,0,0,0,Ice Hockey,Slovenia,26.309431
2853,31,1982-12-05,Female,1.70,Zina Kocher,60.0,0,0,0,0,Biathlon,Canada,20.761246
2854,28,1985-06-14,Female,1.68,Zoe Gillings,65.0,0,0,0,0,Snowboard,Great Britain,23.030045
2856,22,1991-03-01,Male,1.76,Zongyang Jia,68.0,0,0,1,1,Freestyle Skiing,China,21.952479


В качестве критериев для первой модели возьмем рост, вес и общее количество медалей. В фрейме ниже показана закономерность распределения медалей

In [42]:
med_by_gen = data[['gender', 'gold_medals', 'silver_medals', 'bronze_medals', 'total_medals']].groupby('gender').sum()
med_by_gen

,gold_medals,silver_medals,bronze_medals,total_medals
gender,,,,
Female,35,31,32,98
Male,39,43,43,125


In [40]:
cnt = data[['name', 'gender']].groupby('gender').nunique()
cnt

,name
gender,
Female,966
Male,1512


In [45]:
# на одну девушку приходится меньше мелалей, чем на мужчину
med_by_gen.total_medals/cnt.name

gender
Female    0.101449
Male      0.082672
dtype: float64

In [86]:
model_1_inp = ['weight', 'height', 'total_medals']
model_2_inp = ['weight', 'height', 'BMI', 'total_medals']
model_3_inp = ['weight', 'height', 'BMI']
model_out = 'gender'

def model(dataset, input_data: list, output_data: str): 
    inp = dataset[input_data]
    out = dataset[output_data]
    out = out.map({'Male':1, 'Female':0})
    X_train, X_test, Y_train, Y_test = model_selection.train_test_split(inp, out, test_size=0.2)
    
    mod = linear_model.LogisticRegression()
    mod.fit(X_train, Y_train)
    
    Y_test_predicted = mod.predict(X_test)
    
    [tn, fp], [fn, tp] = confusion_matrix(Y_test, Y_test_predicted)
    print(f'Accuracy is {(tn + tp) / (tp + fp + tn + fn)}')
    print(f'Model input: {input_data}\n')
    
    return model

model_1 = model(data ,model_2_inp, 'gender')
model_2 = model(data ,model_1_inp, 'gender')
model_3 = model(data ,model_3_inp, 'gender')

Accuracy is 0.8709677419354839
Model input: ['weight', 'height', 'BMI', 'total_medals']

Accuracy is 0.8487903225806451
Model input: ['weight', 'height', 'total_medals']

Accuracy is 0.844758064516129
Model input: ['weight', 'height', 'BMI']



Вывод: 3-я модель - лучшая 